In [1]:
import concurrent.futures
import time

from tqdm import tqdm


In [2]:
def cpu_bound_task(x):
    print(f"Processing {x} on CPU-bound task")
    return sum(i * i for i in range(1000000 * x))


def io_bound_task(x):
    print(f"Processing {x} on I/O-bound task")
    if x == 2:
        time.sleep(5)
    time.sleep(x)
    return x


def process_pool_example():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = executor.map(cpu_bound_task, range(1, 4))
        for result in results:
            print(f"Process result: {result}")


def thread_pool_example():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(io_bound_task, i) for i in range(1, 4)]
        for future in concurrent.futures.as_completed(futures):
            print(f"Thread result: {future.result()}")


process_pool_example()

Processing 1 on CPU-bound taskProcessing 2 on CPU-bound taskProcessing 3 on CPU-bound task


Process result: 333332833333500000
Process result: 2666664666667000000
Process result: 8999995500000500000


In [3]:
thread_pool_example()

Processing 1 on I/O-bound task
Processing 2 on I/O-bound task
Processing 3 on I/O-bound task
Thread result: 1
Thread result: 3
Thread result: 2


In [4]:
def process_pool_example2():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(io_bound_task, range(1, 4))
        for result in results:
            print(f"Process result: {result}")


process_pool_example2()

Processing 1 on I/O-bound task


Processing 2 on I/O-bound task
Processing 3 on I/O-bound task
Process result: 1
Process result: 2
Process result: 3


In [5]:
def io_bound_task2(x):
    if x == 2:
        time.sleep(10)
    if x == 3:
        time.sleep(9)
    time.sleep(x)
    return x


def process_pool_example3():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = tqdm(executor.map(io_bound_task2, range(1, 4)), total=3)
        for result in results:
            print(f"Process result: {result}")


process_pool_example3()


 33%|███▎      | 1/3 [00:00<00:01,  1.00it/s]

Process result: 1


100%|██████████| 3/3 [00:12<00:00,  4.00s/it]

Process result: 2
Process result: 3


In [6]:
def io_bound_task3(x):
    time.sleep(x)
    return x


def process_pool_example_with_submit():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(io_bound_task3, i) for i in range(1, 4)]
        for future in tqdm(
            concurrent.futures.as_completed(futures), total=len(futures), desc="Processing"
        ):
            result = future.result()
            print(f"Process result: {result}")


process_pool_example_with_submit()


Processing:  33%|███▎      | 1/3 [00:00<00:01,  1.00it/s]

Process result: 1


Processing:  67%|██████▋   | 2/3 [00:02<00:01,  1.00s/it]

Process result: 2


Processing: 100%|██████████| 3/3 [00:03<00:00,  1.00s/it]

Process result: 3


In [7]:
def process_pool_example_with_submit2():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [executor.submit(io_bound_task3, i) for i in range(1, 4)]
        with tqdm(total=len(futures), desc="Processing") as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update()


process_pool_example_with_submit2()


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing: 100%|██████████| 3/3 [00:03<00:00,  1.00s/it]
